<a href="https://colab.research.google.com/github/Pragna235/Decision-Tree-Model---Tennis-Dataset/blob/main/ID3_Decision_Tree_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementation of ID3 Decision Tree Classifier:

1. Entropy of a Set

2. Average Information Entropy of attribute given a set

3. Information Gain (IG) of each attribute and attribute with max IG.

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Find Entropy Function

import numpy as np
def find_entropy(df):
  Class = df.keys()[-1]
  values = df[Class].unique()
  entropy = 0
  for value in values:
    prob = df[Class].value_counts()[value]/len(df[Class])
    entropy += -prob * np.log2(prob)
  return np.float(entropy)

In [19]:
# Find entropy attribute
def find_entropy_attribute(df, attribute):
  Class = df.keys()[-1]
  target_values = df[Class].unique()
  attribute_values = df[attribute].unique()
  avg_entropy = 0
  for value in attribute_values:
    entropy = 0
    for value1 in target_values:
      num = len(df[attribute][df[attribute] == value][df[Class] == value1])
      den = len(df[attribute][df[attribute] == value])
      prob = num/den
      entropy += -prob * np.log2(prob + 0.000001)
    avg_entropy += (den/len(df))*entropy
  return np.float(avg_entropy)

In [20]:
# Find Winner
def find_winner(df):
  IG = []
  for key in df.keys()[:-1]:
    IG.append(find_entropy(df) - find_entropy_attribute(df, key))
  return df.keys()[:-1][np.argmax(IG)]

In [21]:
def get_subtable(df, attribute, value):
  return df[df[attribute] == value].reset_index(drop = True)

In [22]:
def buildtree(df, tree = None):
  node = find_winner(df)
  attvalue = np.unique(df[node])
  Class = df.keys()[-1]
  if tree is None:
    tree = {}
    tree[node] = {}
  for value in attvalue:
    subtable = get_subtable(df,node,value)
    Clvalue, counts = np.unique(subtable[Class], return_counts = True)
    if len(counts) == 1:
      tree[node][value] = Clvalue[0]
    else:
      tree[node][value] = buildtree(subtable)
  return tree

In [23]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/PlayTennis.csv')

In [24]:
tree = buildtree(df)

<ipython-input-18-4217341660d6>:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.float(entropy)
<ipython-input-19-81067c7d688a>:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.float(avg_entropy)
<ipython-input-18-4217341660d6>:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silen

In [25]:
import pprint
pprint.pprint(tree)

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


In [26]:
def predict(inst, tree):
  for node in tree.keys():
    value = inst[node]
    tree = tree[node][value]
    prediction = 0
    if type(tree) is dict:
      prediction = predict(inst, tree)
    else:
      prediction = tree
  return prediction

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/test.csv')

In [ ]:
Y_label = []
for i in range(len(df1)):
  inst = df1.iloc[i,:]
  prediction = predict(inst, tree)
  Y_label.append(prediction)
print(Y_label)

['No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes']


In [ ]:
from sklearn import metrics
print(metrics.classification_report(df1.iloc[:,-1], Y_label))

              precision    recall  f1-score   support

          No       0.25      0.20      0.22         5
         Yes       0.33      0.40      0.36         5

    accuracy                           0.30        10
   macro avg       0.29      0.30      0.29        10
weighted avg       0.29      0.30      0.29        10



In [28]:
print(metrics.accuracy_score(df1.iloc[:,-1],Y_label))

0.3
